In [1]:
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import col, from_json

In [ ]:
spark = SparkSession.builder \
        .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1,org.apache.hadoop:hadoop-client-runtime:3.3.4") \
        .getOrCreate()

In [9]:
class Bronze():
    def __init__(self) -> None:
        self.base_dir = "Apache-Spark-and-Databricks-Stream-Processing-in-Lakehouse-main/Kafka_Practice/data"
        self.BOOTSTRAP_SERVER = "pkc-619z3.us-east1.gcp.confluent.cloud:9092"
        self.JAAS_MODULE = "org.apache.kafka.common.security.plain.PlainLoginModule"
        self.CLUSTER_API_KEY = "WAMFMTSHUDP2D3I3"
        self.CLUSTER_API_SECRET = "VXnP+yvvBC1F5nmO3J5pYEJ3GEwQj/YFE8GLDdmi9WloeaDhjo4AvW9Eoi6AEg2Y"
        self.invoice_schema = """InvoiceNumber string, CreatedTime bigint, StoreID string, PosID string, CashierID string,
                        CustomerType string, CustomerCardNo string, TotalAmount double, NumberOfItems bigint, 
                        PaymentMethod string, TaxableAmount double, CGST double, SGST double, CESS double, 
                        DeliveryType string,
                        DeliveryAddress struct<AddressLine string, City string, ContactNumber string, PinCode string, 
                        State string>,
                        InvoiceLineItems array<struct<ItemCode string, ItemDescription string, 
                            ItemPrice double, ItemQty bigint, TotalValue double>>
                        """

    def ingest_from_kafka(self, start_timestamp=1):
        return (spark.readStream
            .format("kafka")
            .option("kafka.bootstrap.servers", self.BOOTSTRAP_SERVER)
            .option("kafka.security.protocol", "SASL_SSL")
            .option("kafka.sasl.mechanism", "PLAIN")
            .option("kafka.sasl.jaas.config", f"{self.JAAS_MODULE} required username='{self.CLUSTER_API_KEY}' password='{self.CLUSTER_API_SECRET}';")
            .option("subscribe", "invoices")
            .option("maxOffsetsPerTrigger", 10)
            .option("startingTimestamp", start_timestamp)
            .load()
            )
    
    def get_invoices(self, kafka_df: DataFrame):
        return (kafka_df.select(col('key').cast("string").alias("key"),
                                from_json(col("value").cast("string"), self.invoice_schema).alias("value"),
                                "topic", "timestamp")
                                )    
    
    def process(self, start_timestamp=1):
        print("Starting Bronze Stream...", end='')
        kafka_df = self.ingest_from_kafka(start_timestamp)
        invoice_df = self.get_invoices(kafka_df)
        sQuery =  (invoice_df.writeStream
                    .queryName("bronze_query")
                    .format("parquet")
                    .option("checkpointLocation", f"{self.base_dir}/chekpoint/invoices")
                    .outputMode("append")
                    .start("/root/data_engineer2/Apache-Spark-and-Databricks-Stream-Processing-in-Lakehouse-main/Kafka_Practice/data/output"))
        print("Done")
        return sQuery

In [10]:
bronze_kafka = Bronze()
bronze_kafka.process()


Starting Bronze Stream...Done


24/12/12 11:37:19 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


24/12/12 11:37:22 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


In [3]:
df = spark.read.parquet("/root/data_engineer2/Apache-Spark-and-Databricks-Stream-Processing-in-Lakehouse-main/Kafka_Practice/data/output")